 # biopython and sequences from IMGT/HLA downloaded with dbfetch
<p>
I wanted to see if I could download an HLA sequence directly from IMGT/HLA using `dbfetch` and put it into a biopython sequence object. This is what I came up with...
</p>
* documentation
  * [biopython](http://biopython.org/)
  * [requests](http://docs.python-requests.org/en/master/)
  * [dbfetch](http://www.ebi.ac.uk/Tools/dbfetch/)
  * [NamedTemporaryFile](https://docs.python.org/2/library/tempfile.html)

In [1]:
import sys, os
from tempfile import NamedTemporaryFile # part of standard libs
import requests                         # pip install requests
from Bio import SeqIO                   # pip install biopython
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC
from Bio.SeqRecord import SeqRecord

In [2]:
def getHLA(accession):
    '''
    takes an accession ID and returns the sequence record from IMGT/HLA
    '''
    url = ('https://www.ebi.ac.uk/Tools/dbfetch/dbfetch?db=imgthla;id=' +
           accession + ';style=raw'
          )
    try:
        response = requests.get(url)
    except requests.exceptions.RequestException as e:
        print(e)
        sys.exit(1)
    # create a temp file to hold the sequence record
    f = NamedTemporaryFile(mode='w+', delete=False)
    f.write(response.text)
    f.close()
    # read the dbfetch result into a sequence object
    mySeq = SeqIO.read(f.name, "imgt")
    os.unlink(f.name)
    return mySeq

In [3]:
mySeq = getHLA('HLA00001')
print('id = {}'.format(mySeq.id))
print('len = {}'.format(len(mySeq)))
print('description = {}'.format(mySeq.description))
subSeq = mySeq.seq[0:10]
print('1st ten nucleotides = {}'.format(subSeq))

id = HLA00001.1
len = 3503
description = HLA-A*01:01:01:01, Human MHC Class I sequence
1st ten nucleotides = CAGGAGCAGA


In [4]:
print('entire sequence:')
print(mySeq.seq)

entire sequence:
CAGGAGCAGAGGGGTCAGGGCGAAGTCCCAGGGCCCCAGGCGTGGCTCTCAGGGTCTCAGGCCCCGAAGGCGGTGTATGGATTGGGGAGTCCCAGCCTTGGGGATTCCCCAACTCCGCAGTTTCTTTTCTCCCTCTCCCAACCTACGTAGGGTCCTTCATCCTGGATACTCACGACGCGGACCCAGTTCTCACTCCCATTGGGTGTCGGGTTTCCAGAGAAGCCAATCAGTGTCGTCGCGGTCGCTGTTCTAAAGTCCGCACGCACCCACCGGGACTCAGATTCTCCCCAGACGCCGAGGATGGCCGTCATGGCGCCCCGAACCCTCCTCCTGCTACTCTCGGGGGCCCTGGCCCTGACCCAGACCTGGGCGGGTGAGTGCGGGGTCGGGAGGGAAACCGCCTCTGCGGGGAGAAGCAAGGGGCCCTCCTGGCGGGGGCGCAGGACCGGGGGAGCCGCGCCGGGAGGAGGGTCGGGCAGGTCTCAGCCACTGCTCGCCCCCAGGCTCCCACTCCATGAGGTATTTCTTCACATCCGTGTCCCGGCCCGGCCGCGGGGAGCCCCGCTTCATCGCCGTGGGCTACGTGGACGACACGCAGTTCGTGCGGTTCGACAGCGACGCCGCGAGCCAGAAGATGGAGCCGCGGGCGCCGTGGATAGAGCAGGAGGGGCCGGAGTATTGGGACCAGGAGACACGGAATATGAAGGCCCACTCACAGACTGACCGAGCGAACCTGGGGACCCTGCGCGGCTACTACAACCAGAGCGAGGACGGTGAGTGACCCCGGCCCGGGGCGCAGGTCACGACCCCTCATCCCCCACGGACGGGCCAGGTCGCCCACAGTCTCCGGGTCCGAGATCCACCCCGAAGCCGCGGGACTCCGAGACCCTTGTCCCGGGAGAGGCCCAGGCGCCTTTACCCGGTTTCATTTTCAGTTTAGGCCAAAAATCCCCCCGGGTTGGTCGGGGCGGGGCGGGGCTCGG

## Now I wanted to see if I could extract features from the sequence object.

In [5]:
def getFeatures(seqObj, featureType):
    '''
    takes a sequence object and a kind of feature
    returns a list of those features
    '''
    types=[]
    for feature in seqObj.features:
        if feature.type == featureType:
            types.append(feature)
    return types

def getExon(seqObj, num):
    '''
    takes a sequence object with exon features,
    returns another sequence object of only that exon
    '''
    exons = getFeatures(seqObj, 'exon')
    for exon in exons:
        for key, values in exon.qualifiers.items():
            if key == 'number':
                for value in values:
                    if value == str(num):
                        start = exon.location.start
                        end = exon.location.end
                        
    exonSeqObj = SeqRecord(Seq(str(seqObj.seq[start:end]),IUPAC.ambiguous_dna),
                        id='{}, exon {}'.format(seqObj.id, num),
                        name='{}, exon {}'.format(seqObj.name, num),
                        description=("{}, exon {}, start={}, end={}".format(seqObj.description, num, start, end))
                        )
    return exonSeqObj

In [6]:
exon2 = getExon(mySeq, 2)
print(exon2)

ID: HLA00001.1, exon 2
Name: HLA00001, exon 2
Description: HLA-A*01:01:01:01, Human MHC Class I sequence, exon 2, start=503, end=773
Number of features: 0
Seq('GCTCCCACTCCATGAGGTATTTCTTCACATCCGTGTCCCGGCCCGGCCGCGGGG...ACG', IUPACAmbiguousDNA())


In [7]:
print(exon2.seq)

GCTCCCACTCCATGAGGTATTTCTTCACATCCGTGTCCCGGCCCGGCCGCGGGGAGCCCCGCTTCATCGCCGTGGGCTACGTGGACGACACGCAGTTCGTGCGGTTCGACAGCGACGCCGCGAGCCAGAAGATGGAGCCGCGGGCGCCGTGGATAGAGCAGGAGGGGCCGGAGTATTGGGACCAGGAGACACGGAATATGAAGGCCCACTCACAGACTGACCGAGCGAACCTGGGGACCCTGCGCGGCTACTACAACCAGAGCGAGGACG
